In [99]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle
import math
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time
import re
import datetime
import dateutil.parser

In [2]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [3]:
# create splinter browser
def init_browser(noShowBrowser):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=noShowBrowser)
    return browser

In [4]:
browser = init_browser(False)

In [5]:
nba_odds_url = "https://www.actionnetwork.com/nba/live-odds"

In [6]:
browser.visit(nba_odds_url)

In [15]:
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')

In [43]:
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [45]:
df[0].columns = df[0].columns.droplevel()

In [51]:
df_odds = df[0][['SCHEDULEDSCHEDULED','OPEN']]

In [86]:
gameDate = datetime.date.today().strftime("%x")
gameDate

'02/11/19'

In [91]:
#temp = time.strptime(gameDate, '%m/%d/%y')
#temp.isoformat()
date = parser.parse(gameDate).isoformat()
date

'2019-02-11T00:00:00'

In [70]:
roadTeam = []
homeTeam = []
pointSpread = []
for index, row in df_odds.iterrows():
    game = row['SCHEDULEDSCHEDULED']
    roadTeam.append(game[3:6])
    homeTeam.append(game[9:12])
    pointSpread.append(-1.0*float(row['OPEN'].split()[0]))
    #print(row['OPEN'])

In [71]:
pointSpread

[5.5, 4.5, -4.5, -5.0, -9.5]

In [100]:
def getDailyOdds(df_odds):
    roadTeam = []
    homeTeam = []
    pointSpread = []
    gameDate = dateutil.parser.parse(datetime.date.today().strftime("%x")).isoformat()
    for index, row in df_odds.iterrows():
        game = row['SCHEDULEDSCHEDULED']
        roadTeam.append(game[3:6])
        homeTeam.append(game[9:12])
        pointSpread.append(-1.0*float(row['OPEN'].split()[0]))
    oddsDict = {'GameDate' : gameDate,
                'roadTeam' : roadTeam,
                'homeTeam' : homeTeam,
                'pointSpread' : pointSpread
               }
    return oddsDict

In [101]:
temp = getDailyOdds(df_odds)

In [102]:
temp

{'GameDate': '2019-02-11T00:00:00',
 'roadTeam': ['LAL', 'SAS', 'ORL', 'BOS', 'UTA'],
 'homeTeam': ['ATL', 'MEM', 'NOP', 'PHI', 'GSW'],
 'pointSpread': [5.5, 4.5, -4.5, -5.0, -9.5]}

In [96]:
db.nba_odds.insert_one(temp)

In [110]:
def getPointsSpread():
    # create mongo db connection
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    # connect to my nba data db
    db = client.nba_data_db
    temp = db.nba_odds.find().sort('GameDate',pymongo.DESCENDING).limit(1)
    date_index = temp[0]['GameDate']
    last_date = dateutil.parser.parse(date_index)
    last_date = last_date.strftime('%m/%d/%Y')
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    temp[0]['GameDate'] = last_date
    return temp[0]

In [111]:
dailyOdds = getPointsSpread()
dailyOdds

{'GameDate': '02/11/2019',
 'roadTeam': ['LAL', 'SAS', 'ORL', 'BOS', 'UTA'],
 'homeTeam': ['ATL', 'MEM', 'NOP', 'PHI', 'GSW'],
 'pointSpread': [5.5, 4.5, -4.5, -5.0, -9.5]}